# Stock Price Quick Analysis

In [21]:
!pip install requests pandas yfinance investpy lxml beautifulsoup4 tabulate plotly
pip install nbformat --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.8 MB/s eta 0:00:002.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 3.1 MB/s eta 0:00:003.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objs as go

def get_date_from_period(period):
    end_date = datetime.now()
    
    if period == 'ytd':
        start_date = datetime(end_date.year, 1, 1)
    else:
        delta = {
            '1d': 1,
            '5d': 5,
            '1m': 30,
            '6m': 182,
            '1y': 365,
            '2y': 365*2,
            '3y': 365*3,
            '5y': 365*5,
            '10y': 365*10,
            '20y': 365*20
        }[period]
        start_date = end_date - timedelta(days=delta)
    
    return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

def get_stock_performance(symbol, start_date, end_date=None):
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    
    if hist.empty:
        return None

    start_price = round(hist['Close'].iloc[0], 1)
    end_price = round(hist['Close'].iloc[-1], 1)
    percent_change = round(((end_price - start_price) / start_price) * 100, 1)

    return {
        "symbol": symbol,
        "start_date": start_date,
        "end_date": end_date,
        "start_price": start_price,
        "end_price": end_price,
        "percent_change": percent_change,
        "history": hist
    }

def print_stock_performance(results):
    if not results:
        print("No data available for the specified period.")
        return

    df = pd.DataFrame(results)
    df['percent_change'] = df['percent_change'].apply(lambda x: f"{x:.1f}%")
    df = df.sort_values(by='percent_change', ascending=False)
    print(df[['symbol', 'start_date', 'end_date', 'start_price', 'end_price', 'percent_change']])

def plot_stock_performance_interactive(results, price_type='Close', normalize=False):
    if not results:
        print("No data available to plot.")
        return

    fig = go.Figure()
    
    for result in results:
        hist = result['history']
        if normalize:
            hist = hist.copy()
            hist[price_type] = hist[price_type] / hist[price_type].iloc[0]
        fig.add_trace(go.Scatter(x=hist.index, y=hist[price_type], mode='lines', name=f'{result["symbol"]} ({price_type} Price)'))
    
    fig.update_layout(
        title=f"Stock Performance from {results[0]['start_date']} to {results[0]['end_date']}",
        xaxis_title='Date',
        yaxis_title=f'{"Normalized " if normalize else ""}{price_type} Price',
        hovermode='x unified'
    )
    
    fig.show()

def get_and_print_stock_performance(symbols, period=None, start_date=None, end_date=None, normalize=False):
    if isinstance(symbols, str):
        symbols = [symbols]

    if period and not start_date:
        start_date, end_date = get_date_from_period(period)
    elif start_date and not end_date:
        end_date = datetime.now().strftime('%Y-%m-%d')
    
    results = []
    for symbol in symbols:
        result = get_stock_performance(symbol, start_date, end_date)
        if result:
            results.append(result)

    print_stock_performance(results)
    plot_stock_performance_interactive(results, price_type='Close', normalize=normalize)

In [20]:
# Example usage:
symbols = ['AAPL', 'MSFT', 'TSLA', 'BTC-USD', 'TQQQ']
period = '5y'  # Options: 'ytd', '1d', '5d', '1m', '6m', '1y', '2y', '3y', '5y', '10y', '20y'
start_date = None #'2024-01-01'  # '2021-01-01' can be specified here to override period
end_date = None  # Example: '2023-12-31'

get_and_print_stock_performance(symbols, period=period, start_date=start_date, end_date=end_date, normalize=True)

    symbol  start_date    end_date  start_price  end_price percent_change
3  BTC-USD  2019-07-05  2024-07-03      10978.5    62029.0         465.0%
4     TQQQ  2019-07-05  2024-07-03         16.0       77.3         383.1%
0     AAPL  2019-07-05  2024-07-03         49.3      220.3         346.9%
1     MSFT  2019-07-05  2024-07-03        130.7      459.3         251.4%
2     TSLA  2019-07-05  2024-07-03         15.5      231.3        1392.3%
